In [1]:
%run ../../preamble.py

# Simulation study: the well specified case

In [19]:
# Setting the true model
f1, f2 = bs.loss_model("Gamma", ["μ1", "λ1"]), bs.loss_model("Lomax", ["α2", "σ2"])
f_true = bs.spliced_loss_model(f1, f2, "continuous")
parms_true = np.array([1/2, 1, 2.5, 3, 1.5])
f_true.set_ppf(), f_true.set_pdf(),f_true.set_cdf()

# We set the priority to the 90% quantile and the limit to the 0.99% quantile
P, L = f_true.ppf(parms_true, 0.9), f_true.ppf(parms_true, 0.99)
premiums = f_true.PP(parms_true), f_true.XOLP(parms_true, P, L)
expo = 250
PnLs = np.array(f_true.PnL(parms_true, P, L, expo, premiums, safety_loadings = [0.05, 0.05], n_sim = 10))

We are interested in the estimations of the extreme quantile of the claim size distribution (of order 0.95, 0.99, 0.995) and the quantile of the aggregate losses over one year with a XOL reinsurance agreement

In [20]:
true_VaRs = [f_true.ppf(parms_true, prob) for prob in [0.95, 0.99, 0.995]]
true_cap = np.quantile(PnLs, [0.005, 0.01, 0.05])
true_cap

array([-62.40601976, -59.36290652, -35.01800061])

In [23]:
# Model for the bulk distribution
body_model_names = ["Exp", "Gamma", "Weibull", "Inverse-Gaussian", "Lognormal", "Inverse-Weibull", "Inverse-Gamma"]
body_model_param_names = [ ["λ1"], ["r1", "m1"], ["k1", "β1"],["μ1", "λ1"],
                          ["μ1", "σ1"], ["k1", "β1"], ["r1", "m1"]]

# Prior distributions over the parameters of the bulk distribution
body_model_priors= [ 
    [bs.prior_model('gamma',body_model_param_names[0][0], 1, 1)], 
     [bs.prior_model('gamma',body_model_param_names[1][0], 1, 1), bs.prior_model('gamma',body_model_param_names[1][1], 1, 1)],
    [bs.prior_model('gamma',body_model_param_names[2][0], 1, 1), bs.prior_model('gamma',body_model_param_names[2][1], 1, 1)],
    [bs.prior_model('gamma',body_model_param_names[3][0], 1, 1), bs.prior_model('gamma',body_model_param_names[3][1], 1, 1)],
    [bs.prior_model('normal',body_model_param_names[4][0], 0, 0.5), bs.prior_model('gamma',body_model_param_names[4][1], 1, 1)],
     [bs.prior_model('gamma',body_model_param_names[5][0], 1, 1), bs.prior_model('gamma',body_model_param_names[5][1], 1, 1)], 
    [bs.prior_model('gamma',body_model_param_names[6][0], 1, 1), bs.prior_model('gamma',body_model_param_names[6][1], 1, 1)]
]

# Model for the tail of the distribution
tail_model_names = ["Weibull", "Lognormal", "Log-Logistic", "Lomax", "Burr", "Pareto-Tail", "GPD-Tail", "Inverse-Gamma", "Inverse-Weibull"]

tail_model_param_names = [["k2", "β2"], ["μ2", "σ2"], ["β2", "σ2"], ["α2", "σ2"], ["α2", "β2", "σ2"], ["α2"], ["ξ2","σ2"], ["r2", "m2"], ["k2", "β2"]]

# Prior distributions over the parameters of the bulk distribution
tail_model_priors= [
                [bs.prior_model('gamma',tail_model_param_names[0][0], 1, 1), bs.prior_model('gamma',tail_model_param_names[0][1], 1, 1)],
                [bs.prior_model('normal',tail_model_param_names[1][0], 0, 0.5), bs.prior_model('gamma',tail_model_param_names[1][1], 1, 1)],
                [bs.prior_model('gamma',tail_model_param_names[2][0], 1, 1), bs.prior_model('gamma',tail_model_param_names[2][1], 1, 1)],
                [bs.prior_model('gamma',tail_model_param_names[3][0], 1, 1), bs.prior_model('gamma',tail_model_param_names[3][1], 1, 1)],
                [bs.prior_model('gamma',tail_model_param_names[4][0], 1, 1), bs.prior_model('gamma',tail_model_param_names[4][1], 1, 1), bs.prior_model('gamma',tail_model_param_names[4][2], 1, 1)],
                [bs.prior_model('gamma',tail_model_param_names[5][0], 1, 1)],
                [bs.prior_model('gamma',tail_model_param_names[6][0], 1, 1), bs.prior_model('gamma',tail_model_param_names[6][1], 1, 1)],
                [bs.prior_model('gamma',tail_model_param_names[7][0], 1, 1), bs.prior_model('gamma',tail_model_param_names[7][1], 1, 1)],
                [bs.prior_model('gamma',tail_model_param_names[8][0], 1, 1), bs.prior_model('gamma',tail_model_param_names[8][1], 1, 1)]
]

γ_prior = bs.prior_model('gamma',"γ", 1, 1)

#Splicing model type
splicing_types = ["continuous"]

# Setting the models
fs, f_names, prior_spliced_model = [], [], []
for i in range(len(body_model_names)):
    for j in range(len(tail_model_names)):
        for splicing_type in splicing_types:
            f1, f2 =  bs.loss_model(body_model_names[i], body_model_param_names[i]), bs.loss_model(tail_model_names[j], tail_model_param_names[j])
            fs.append(bs.spliced_loss_model(f1 , f2, splicing_type))
            f_names.append(body_model_names[i] +"_"+ tail_model_names[j]+"_"+splicing_type)
            if splicing_type == "disjoint": 
                prior_spliced_model.append(bs.independent_priors(body_model_priors[i] + tail_model_priors[j] + [γ_prior, p_prior]))
            else:
                prior_spliced_model.append(bs.independent_priors(body_model_priors[i] + tail_model_priors[j] + [γ_prior]))  
for f in fs:
    f.set_ppf(), f.set_cdf(), f.set_pdf() 
f_spliced_dic = dict(zip(f_names, fs))

In [24]:
nobs, n_sim = 250, 2
Xs = [f_true.sample(parms_true, nobs) for k in range(n_sim)]
popSize, ρ, c, n_step_max, err, paralell, n_proc, verbose = 50, 1/2, 0.99, 25, 1e-6, False, 4, False
def fit_spliced_models(k):
    print("Sim #"+str(k))
    res = []
    for i in range(len(fs)):
        trace, log_marg, DIC, WAIC = bs.smc(Xs[k], fs[i], popSize, prior_spliced_model[i], ρ, c,n_step_max, err, paralell, 4, verbose)
        VaRs = [fs[i].ppf(trace.mean().values, prob) for prob in [0.95, 0.99, 0.995]]
        premiums = fs[i].PP(trace.mean().values), fs[i].XOLP(trace.mean().values, P, L)
        PnLs = np.array(fs[i].PnL(trace.mean().values, P, L, expo, premiums, safety_loadings = [0.05, 0.05], n_sim = 10))
        caps = np.quantile(PnLs, [0.005, 0.01, 0.05])
        res.append(np.array([k, f_names[i], log_marg] + VaRs + caps.tolist()))
    return(res)
%time res = Parallel(n_jobs=2)(delayed(fit_spliced_models)(k) for k in range(n_sim))


Wall time: 6min 42s


In [33]:
dfs = pd.concat([pd.DataFrame(re, columns = ["sim","model_name", "log_marg", "q95", "q99", "q995", "cap005", "cap01",  "cap05"]) for re in res])
dfs

,sim,model_name,log_marg,q95,q99,q995,cap005,cap01,cap05
0,0,Exp_Weibull_continuous,-161.27223905304925,4.695724697967307,11.06700651121173,14.650759538881175,-41.7176605085789,-41.105015315428766,-36.20385377022772
1,0,Exp_Lognormal_continuous,-167.33192102527025,6.509880989824524,24.325947123315927,39.43031501307814,-146.1859504965678,-145.70077578533076,-141.8193780954342
2,0,Exp_Log-Logistic_continuous,-170.52970061092185,7.717863326206498,42.62643855668102,87.84758236402013,3553.455716732908,3555.794720240338,3574.506748299776
3,0,Exp_Lomax_continuous,-169.58262954107968,5.450404927387353,20.230886121908043,34.77589203271236,-285.3989707214815,-280.2616363259678,-239.1629611618581
4,0,Exp_Burr_continuous,-164.40344703718006,5.576072936719356,22.59273301774556,39.457716383139996,-3116.043550604115,-3003.849639517166,-2106.298350821575
...,...,...,...,...,...,...,...,...,...
58,1,Inverse-Gamma_Burr_continuous,-176.666622385216,5.525554769292285,22.16117493958336,38.80710934822673,-587.9039081672743,-573.8646994926128,-461.5510300953214
59,1,Inverse-Gamma_Pareto-Tail_continuous,-271.88090816211485,4.357460881010411,10.54019023172259,15.39376270898174,-83.91625411510373,-81.2485855454997,-59.90723698866749
60,1,Inverse-Gamma_GPD-Tail_continuous,-185.1177798277091,5.180802276072427,25.504774470164193,49.93738897951516,1270.6351120553627,1271.9558566111527,1282.5218130574726
61,1,Inverse-Gamma_Inverse-Gamma_continuous,-220.4335291534946,4.500895300907359,21.106450986962955,40.62556726580624,-426.16382034422475,-379.16443770793165,-3.1693766175870905
